# RAG: Procurement Sourcing Tool

## Project Goal

This project demonstrates a Supplier & Procurement Intelligence Sourcing Tool, based on RAG system built with OpenAI embeddings, FAISS for semantic search, and synthetic procurement data.

The workflow begins with the generation of realistic supplier documents such as capability decks (PDFs summarizing a supplier’s products, certifications, and lead times), contracts with key clauses (liability caps, Incoterms), audit reports (ISO/ROHS compliance), PO/ASN histories (used to calculate On-Time Delivery performance), meeting notes, and pricing tables.

These files are chunked into meaningful units — contract clauses, capability sections, and table rows — then embedded using OpenAI’s text-embedding-3-small model. The embeddings are stored in a FAISS index, alongside supplier metadata like region, category, certifications, and ESG flags. A lightweight natural language filter extractor enriches search by applying constraints (e.g., “ISO 13485 suppliers in ASEAN with < 4-week lead time”), and retrieved results are fed into an OpenAI LLM (gpt-4o-mini) to generate concise, context-grounded answers.

### Products & Parts in Scope

The synthetic dataset covers typical industrial components often sourced globally:


*   Impellers (316L stainless steel, sizes 50mm & 75mm)
*   Bearings (type 6205)
*   Pumps
*   Valves

These represent common mechanical parts with varied suppliers, lead times, and quality compliance requirements.

### Procurement KPIs

The project uses common procurement performance metrics:
*   Lead Time: The time between placing an order and receiving the goods.
*   On-Time Delivery (OTD): Percentage of deliveries arriving as promised (e.g., OTD = 95% means 95 out of 100 orders were delivered on time).
*   Audit Score: Result of quality or compliance audits (e.g., ISO certifications, ROHS compliance).
*   Supplier Rating: Overall performance score (synthetic, 1-5 scale).
*   Pricing & Cost: Unit prices in USD for each SKU.

### Synthetic Data Structure

The synthetic dataset is generated in multiple files/folders:


*   Supplier Master (suppliers.csv)
	- supplier_id, name, region, categories, certifications, esg_flags, rating
*   Capability Decks (PDFs)
	- Supplier profile (region, certifications, ESG flags)
	- Claimed lead times (standard/custom parts)
	- Historical OTD by quarter
*   Contracts (PDFs)
	- Contract ID, supplier details
	- Clauses: Scope, Pricing, Payment Terms, Liability Cap, Incoterms, Quality, Lead Time
*   Audit Reports (audit_reports.csv)
	- supplier_id, iso13485, iso9001, rohs, last_audit, score
*   PO/ASN Histories (po_asn.csv)
	- po_id, supplier_id, sku, category, qty, order_date, promised_date, shipped_date, asn_id, region
	- Used to calculate OTD by quarter
*   Notes (text files)
	- Informal meeting notes, supplier risks, improvement actions
*   Pricing Table (pricing.csv)
	- supplier_id, sku, part, price_usd, lead_time_weeks, region

### How This Helps Procurement

Procurement professionals often spend hours digging through contracts, supplier brochures, audit reports, and spreadsheets. This tool centralizes all that information into a semantic search engine in order to:

*   Quickly find alternative suppliers by part, region, or certification.
*   Check contract clauses like liability caps or Incoterms without reading entire PDFs.
*   Monitor supplier OTD trends and audit scores to evaluate reliability.
*   Compare prices and lead times across suppliers to optimize sourcing.
*   Apply ESG filters (e.g., conflict-free, low emissions) to align with sustainability goals.

By automating retrieval and summarization, procurement teams save time, reduce risk, and make better, data-backed sourcing decisions. Instead of manually opening dozens of PDFs, spreadsheets, and audit logs, category managers or supplier quality specialists can ask natural-language questions and instantly surface compliant alternative suppliers, lead-time risks, or contract terms. It reduces time-to-source, highlights suppliers with strong past OTD performance, and flags contract risks without requiring hours of document review. By filtering across ESG compliance and certifications, it also enables faster alignment with sustainability and regulatory requirements. Ultimately, this system empowers procurement professionals to make faster, evidence-backed sourcing decisions, improves supplier risk management, and frees teams from repetitive data lookup tasks so they can focus on strategic supplier relationships.


## Setup

In [3]:
# Mounting to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd "your-path-here"

In [ ]:
# Uninstall potentially incompatible preinstalls, then install a stable stack.
%%capture
!pip -q uninstall -y numpy numexpr pandas faiss-cpu || true
!pip -q install --upgrade --force-reinstall \
  numpy==1.26.4 \
  numexpr==2.8.7 \
  pandas==2.2.2 \
  faiss-cpu==1.8.0.post1 \
  langchain==0.2.14 \
  langchain-openai==0.1.22 \
  langchain-community==0.2.12 \
  rank-bm25==0.2.2 \
  pypdf==4.3.1 \
  reportlab==4.2.2 \
  python-dotenv==1.0.1


In [ ]:
# Colab runtime restart
import os, sys
print("Rebooting Colab runtime to finalize environment...")
os.kill(os.getpid(), 9)

In [ ]:
import numpy as np, pandas as pd
print("NumPy:", np.__version__)    # expect 1.26.4
print("pandas:", pd.__version__)   # expect 2.2.2

NumPy: 1.26.4
pandas: 2.2.2


In [ ]:
import os, re, random, json, datetime as dt
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
from pypdf import PdfReader
from reportlab.lib.pagesizes import LETTER
from reportlab.pdfgen import canvas

# RAG bits
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# BM25
from rank_bm25 import BM25Okapi

import json, textwrap
from textwrap import fill
from typing import List, Tuple
import re
from textwrap import fill

In [ ]:
# Set your OpenAI API Key

os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here

In [6]:
# Settings container
class Settings:
    data_dir    = "data"
    index_dir   = "faiss_store"
    embed_model = "text-embedding-3-small"
    chat_model  = "gpt-4o-mini"
    seed        = 42

SET = Settings()
random.seed(SET.seed)

In [7]:
# Folders
DATA = Path(SET.data_dir); RAW = DATA / "raw"
for sub in ["suppliers","contracts","audits","po_asn","notes","pricing","capability"]:
    (RAW/sub).mkdir(parents=True, exist_ok=True)
Path(SET.index_dir).mkdir(exist_ok=True)

## Generate synthetic data

In [8]:
# Helper to write simple PDFs
def write_pdf(path: Path, title: str, lines: list[str]):
    c = canvas.Canvas(str(path), pagesize=LETTER)
    width, height = LETTER
    y = height - 72
    c.setTitle(title)
    c.setFont("Helvetica-Bold", 16); c.drawString(72, y, title); y -= 24
    c.setFont("Helvetica", 10)
    for line in lines:
        if y < 72:
            c.showPage(); y = height - 72; c.setFont("Helvetica", 10)
        c.drawString(72, y, line); y -= 14
    c.save()

# Master vocab
REGIONS = ["NA","EU","ASEAN","INDIA","CHINA"]
CATS    = ["impellers","bearings","pumps","valves"]
SKUS    = ["IMP-75","IMP-50","BRG-6205","VAL-20","PMP-100"]
CERTS_POOL = [["ISO9001"],["ISO13485"],["ROHS"],["ISO9001","ROHS"],["ISO13485","ROHS"]]
ESG_POOL = [[], ["low_emissions"], ["conflict_free"], ["recycled_materials"],
            ["low_emissions","recycled_materials"]]

# --- Suppliers master with ESG & risk + std lead ---
suppliers = []
for i in range(12):
    std_lt = random.randint(2, 8)
    suppliers.append({
        "supplier_id": f"S{i+1001}",
        "name": f"Acme-{i}",
        "region": random.choice(REGIONS),
        "categories": "|".join(random.sample(CATS, k=random.randint(1,2))),
        "certifications": "|".join(random.choice(CERTS_POOL)),
        "rating": round(random.uniform(3.0, 5.0), 1),
        "std_lead_weeks": std_lt,
        "esg_flags": "|".join(random.choice(ESG_POOL)) or "none",
        "risk_score": random.randint(10, 90)
    })
sup_df = pd.DataFrame(suppliers)
sup_df.to_csv(DATA/"suppliers.csv", index=False)

# --- Capability table per SKU (lead time per SKU, capacity, MOQ) ---
cap_rows=[]
for s in suppliers:
    for sku in random.sample(SKUS, k=random.randint(2,4)):
        lt = max(2, s["std_lead_weeks"] + random.randint(-1, 2))
        cap_rows.append({
            "supplier_id": s["supplier_id"], "sku": sku,
            "capability": f"Manufacturing of {sku}",
            "capacity_week": random.randint(50,500),
            "moq": random.randint(10,100),
            "lead_time_weeks": lt,
            "category": random.choice(CATS)
        })
cap_df = pd.DataFrame(cap_rows)
cap_df.to_csv(RAW/"capability"/"capability.csv", index=False)

# --- Capability deck PDFs (human readable) ---
for s in suppliers:
    lines = [
        f"Supplier: {s['name']} ({s['supplier_id']})",
        f"Region: {s['region']}  Categories: {s['categories'].replace('|', ', ')}",
        f"Certifications: {s['certifications'].replace('|', ', ')}",
        f"ESG: {s['esg_flags'].replace('|', ', ')}  Risk Score: {s['risk_score']}",
        "Capabilities:",
        "- CNC machining, cleanroom packaging",
        f"Standard Lead Time: {s['std_lead_weeks']} weeks",
        f"OTD last quarter: {random.randint(88,99)}%"
    ]
    write_pdf(RAW/"suppliers"/f"{s['supplier_id']}_cap.pdf", f"Capability Deck {s['name']}", lines)

# --- Contracts PDFs with clear definitions ---
CLAUSES = [
    ("Scope", "Supply of parts and components. Definition: Scope = what products/services are covered."),
    ("Pricing", "Quarterly review. Definition: Explains how/when prices can change."),
    ("Payment Terms", "Net 45 days. Definition: Buyer pays within 45 days."),
    ("Liability Cap", "120% of 12m order value. Definition: Limits supplier’s maximum financial responsibility."),
    ("Incoterms", "DDP Buyer Warehouse. Definition: Supplier covers shipping, customs, duties to buyer site."),
    ("Quality", "ISO13485 and ROHS required. Definition: Supplier must keep these certifications."),
    ("Lead Time", "Standard lead time is 4 weeks. Definition: Time from order to delivery.")
]
for s in suppliers:
    lines = [f"Contract for {s['name']} ({s['supplier_id']})",""]
    for h,b in CLAUSES: lines += [f"{h}:", b, ""]
    write_pdf(RAW/"contracts"/f"{s['supplier_id']}_contract.pdf", f"Contract {s['name']}", lines)

# --- Audits CSV ---
audits=[]
for s in suppliers:
    certs = s["certifications"].split("|")
    audits.append({
        "supplier_id": s["supplier_id"],
        "iso13485": int("ISO13485" in certs),
        "iso9001": int("ISO9001" in certs),
        "rohs": int("ROHS" in certs),
        "last_audit": (dt.date.today() - dt.timedelta(days=random.randint(50,300))).isoformat(),
        "score": random.randint(72,98)
    })
audit_df = pd.DataFrame(audits)
audit_df.to_csv(RAW/"audits"/"audit.csv", index=False)

# --- Pricing CSV (align with capability lead time) ---
pricing=[]
for _, r in cap_df.iterrows():
    pricing.append({
        "supplier_id": r["supplier_id"], "sku": r["sku"],
        "price_usd": round(random.uniform(10,300),2),
        "lead_time_weeks": int(r["lead_time_weeks"]),   # same as capability per SKU
        "region": sup_df.loc[sup_df.supplier_id==r["supplier_id"], "region"].iloc[0],
        "category": r["category"]
    })
pricing_df = pd.DataFrame(pricing)
pricing_df.to_csv(RAW/"pricing"/"pricing.csv", index=False)

# --- PO/ASN lines (per PO, all fields filled) ---
po_rows=[]
for i in range(200):
    s = random.choice(suppliers)
    sku = random.choice(SKUS)
    po_date = dt.date.today() - dt.timedelta(days=random.randint(30, 200))
    promised = po_date + dt.timedelta(days=7*max(2, s["std_lead_weeks"] + random.randint(-1, 1)))
    shipped  = po_date + dt.timedelta(days=random.randint(5, 7*max(1, s["std_lead_weeks"])))
    delivered= shipped + dt.timedelta(days=random.randint(1,7))
    asn_date = shipped - dt.timedelta(days=random.randint(0,2))
    on_time  = int(delivered <= promised)
    po_rows.append({
        "po_id": f"PO-{10000+i}",
        "supplier_id": s["supplier_id"],
        "sku": sku,
        "po_date": po_date.isoformat(),
        "promised_date": promised.isoformat(),
        "shipped_date": shipped.isoformat(),
        "asn_date": asn_date.isoformat(),
        "delivery_date": delivered.isoformat(),
        "qty": random.randint(10,200),
        "on_time": on_time
    })
po_df = pd.DataFrame(po_rows)
po_df.to_csv(RAW/"po_asn"/"po_asn.csv", index=False)

# --- OTD summary per supplier (last 90 days) ---
po_df["delivery_date"] = pd.to_datetime(po_df["delivery_date"])
cutoff = pd.Timestamp.today().normalize() - pd.Timedelta(days=90)
otd = (po_df[po_df["delivery_date"]>=cutoff]
       .groupby("supplier_id")["on_time"].mean().reset_index()
       .rename(columns={"on_time":"otd_90d"}))
otd["otd_90d"] = (otd["otd_90d"]*100).round(1)
otd.to_csv(RAW/"po_asn"/"otd_summary.csv", index=False)

print("Synthetic data generated with full fields (capability per SKU, PO/ASN, OTD, ESG, risk).")

Synthetic data generated with full fields (capability per SKU, PO/ASN, OTD, ESG, risk).


## Chunking → LangChain Documents

In [9]:
# PDF reader with error handling
def read_pdf_text(path: Path) -> str:
    try:
        return "\n".join(page.extract_text() or "" for page in PdfReader(str(path)).pages)
    except Exception as e:
        print(f"[Warning] Could not read PDF {path}: {e}")
        return ""

# Utility: CSV reader
def safe_read_csv(path: Path):
    try:
        return pd.read_csv(path)
    except Exception as e:
        print(f"[Warning] Could not read CSV {path}: {e}")
        return pd.DataFrame()

# Utility: ensure consistent metadata keys
def ensure_metadata(md: dict, required_keys: list) -> dict:
    for k in required_keys:
        if k not in md:
            md[k] = None
    return md

# Regex to grab "N weeks"
LT_RE = re.compile(r"(\d+)\s*weeks", re.I)
LT_CAP_RE = re.compile(r"lead\s*times?:\s*(\d+)\s*weeks", re.I)

# Capability decks → one doc (keep std lead)
def chunk_cap_pdf(path: Path, sid: str) -> list[Document]:
    txt = read_pdf_text(path)
    md = {"supplier_id":sid, "doctype":"capability_deck"}
    m = LT_CAP_RE.search(txt)
    if m: md["lead_time_weeks"]=int(m.group(1))
    md = ensure_metadata(md, ["supplier_id", "doctype", "lead_time_weeks"])
    return [Document(page_content=txt, metadata=md)]

# Contracts → per-clause docs
def chunk_contract_pdf(path: Path, sid: str) -> list[Document]:
    txt = read_pdf_text(path)
    lines=[ln.strip() for ln in txt.split("\n") if ln.strip()]
    docs=[]
    for ln in lines:
        md={"supplier_id":sid,"doctype":"contract"}
        if "lead time" in ln.lower():
            m=LT_RE.search(ln)
            if m: md["lead_time_weeks"]=int(m.group(1))
        md = ensure_metadata(md, ["supplier_id", "doctype", "lead_time_weeks"])
        docs.append(Document(page_content=ln, metadata=md))
    return docs

# Capability table → one doc per SKU with LT
def chunk_cap_table(path: Path) -> list[Document]:
    df=safe_read_csv(path); docs=[]
    for _,r in df.iterrows():
        text=(f"Capability: supplier {r.supplier_id} sku {r.sku} capability {r.capability} "
              f"capacity_week {r.capacity_week} moq {r.moq} lead_time_weeks {int(r.lead_time_weeks)} "
              f"category {r.category}")
        md = {
            "supplier_id":r.supplier_id, "doctype":"capability",
            "sku":r.sku, "lead_time_weeks":int(r.lead_time_weeks),
            "category":r.category
        }
        md = ensure_metadata(md, ["supplier_id", "doctype", "sku", "lead_time_weeks", "category"])
        docs.append(Document(page_content=text, metadata=md))
    return docs

# Pricing → one doc per row
def chunk_pricing(path: Path) -> list[Document]:
    df=safe_read_csv(path); docs=[]
    for _,r in df.iterrows():
        text=(f"Pricing: supplier {r.supplier_id} sku {r.sku} price_usd {r.price_usd} "
              f"lead_time_weeks {int(r.lead_time_weeks)} region {r.region} category {r.category}")
        md = {
            "supplier_id":r.supplier_id,"doctype":"pricing","sku":r.sku,
            "lead_time_weeks":int(r.lead_time_weeks),"region":r.region,"category":r.category
        }
        md = ensure_metadata(md, ["supplier_id", "doctype", "sku", "lead_time_weeks", "region", "category"])
        docs.append(Document(page_content=text, metadata=md))
    return docs

# Audit → compliance row
def chunk_audit(path: Path) -> list[Document]:
    df=safe_read_csv(path); docs=[]
    for _,r in df.iterrows():
        text=f"Audit {r.supplier_id} ISO13485={int(r.iso13485)} ROHS={int(r.rohs)} score={int(r.score)}"
        md = {
            "supplier_id":r.supplier_id,"doctype":"audit","iso13485":int(r.iso13485),
            "rohs":int(r.rohs),"score":int(r.score)
        }
        md = ensure_metadata(md, ["supplier_id", "doctype", "iso13485", "rohs", "score"])
        docs.append(Document(page_content=text, metadata=md))
    return docs

# PO/ASN lines → keep concise; OTD summary → explicit percentage
def chunk_po_asn(path_po: Path, path_otd: Path) -> list[Document]:
    docs=[]
    po = safe_read_csv(path_po)
    for _,r in po.iterrows():
        text=(f"PO {r.po_id} supplier {r.supplier_id} sku {r.sku} qty {r.qty} promised {r.promised_date} "
              f"delivered {r.delivery_date} on_time {int(r.on_time)}")
        md = {
            "supplier_id": r.supplier_id, "doctype":"po_asn", "sku": r.sku, "on_time": int(r.on_time)
        }
        md = ensure_metadata(md, ["supplier_id", "doctype", "sku", "on_time"])
        docs.append(Document(page_content=text, metadata=md))
    otd = safe_read_csv(path_otd)
    for _,r in otd.iterrows():
        text=(f"OTD supplier {r.supplier_id} last_90d {float(r.otd_90d)}%")
        md = {
            "supplier_id": r.supplier_id, "doctype":"otd", "otd_90d": float(r.otd_90d)
        }
        md = ensure_metadata(md, ["supplier_id", "doctype", "otd_90d"])
        docs.append(Document(page_content=text, metadata=md))
    return docs

# Supplier profile (with ESG & risk) → one doc per supplier
def chunk_profiles(path: Path) -> list[Document]:
    df=safe_read_csv(path); docs=[]
    for _,r in df.iterrows():
        text=(f"Supplier profile {r.supplier_id} {r.name} region {r.region} "
              f"categories {r.categories} certs {r.certifications} "
              f"std_lead_weeks {int(r.std_lead_weeks)} ESG {r.esg_flags} risk_score {int(r.risk_score)}")
        md = {
            "supplier_id": r.supplier_id, "doctype":"profile",
            "region": r.region, "esg_flags": r.esg_flags, "risk_score": int(r.risk_score)
        }
        md = ensure_metadata(md, ["supplier_id", "doctype", "region", "esg_flags", "risk_score"])
        docs.append(Document(page_content=text, metadata=md))
    return docs

# Build entire corpus
def build_docs():
    docs=[]
    for p in (RAW/"suppliers").glob("*.pdf"):
        sid=p.stem.split("_")[0]; docs+=chunk_cap_pdf(p,sid)
    for p in (RAW/"contracts").glob("*.pdf"):
        sid=p.stem.split("_")[0]; docs+=chunk_contract_pdf(p,sid)
    docs += chunk_cap_table(RAW/"capability"/"capability.csv")
    docs += chunk_pricing(RAW/"pricing"/"pricing.csv")
    docs += chunk_audit(RAW/"audits"/"audit.csv")
    docs += chunk_po_asn(RAW/"po_asn"/"po_asn.csv", RAW/"po_asn"/"otd_summary.csv")
    docs += chunk_profiles(DATA/"suppliers.csv")
    return docs

docs = build_docs()
len(docs)

504

## Build FAISS + prepare BM25 corpus

In [10]:
# FAISS (OpenAI embeddings) with index persistence
emb = OpenAIEmbeddings(model=SET.embed_model)
import os
faiss_index_path = os.path.join(SET.index_dir, "index.faiss")
if os.path.exists(faiss_index_path):
    vs = FAISS.load_local(SET.index_dir, embeddings=emb, allow_dangerous_deserialization=True)
    print("FAISS index loaded from disk:", SET.index_dir)
else:
    vs = FAISS.from_documents(docs, emb)
    vs.save_local(SET.index_dir)
    print("FAISS index saved:", SET.index_dir)

# BM25 setup: keep tokenized corpus + backpointer to Document
def _tokenize(text: str) -> list[str]:
    # whitespace + lowercase; good enough for synthetic data
    return re.findall(r"\b\w+\b", text.lower())

bm25_texts = [d.page_content for d in docs]
bm25_tokens = [_tokenize(t) for t in bm25_texts]
bm25 = BM25Okapi(bm25_tokens)
print("BM25 corpus ready:", len(bm25_texts), "docs")

FAISS index loaded from disk: faiss_store
BM25 corpus ready: 504 docs


## Lightweight NL Query → filters (region, certs, lead time, ESG, risk) & pass function

In [11]:
def extract_filters(q: str) -> dict:
    ql=q.lower(); f={}
    for r in ["asean","eu","na","india","china"]:
        if r in ql: f["region"] = "ASEAN" if r=="asean" else r.upper()
    certs=[]
    if "iso 13485" in ql or "iso13485" in ql: certs.append("ISO13485")
    if "rohs" in ql: certs.append("ROHS")
    if certs: f["certs"]=certs
    # Lead time
    m = re.search(r"(?:<=|<|under|within)\s*(\d+)\s*(?:weeks?|wks?)", ql)
    if m: f["max_lt"]=int(m.group(1))
    m2 = re.search(r"(\d+)\s*(?:weeks?|wks?)\s*(?:or\s*less|max)", ql)
    if "max_lt" not in f and m2: f["max_lt"]=int(m2.group(1))
    # Price filters
    m = re.search(r"price\s*(?:<=|<|under|below)\s*\$?(\d+\.?\d*)", ql)
    if m: f["max_price"] = float(m.group(1))
    m = re.search(r"price\s*(?:>=|>|over|above)\s*\$?(\d+\.?\d*)", ql)
    if m: f["min_price"] = float(m.group(1))
    # Audit score filters
    m = re.search(r"audit\s*score\s*(?:>=|>|over|above)\s*(\d+)", ql)
    if m: f["min_audit_score"] = int(m.group(1))
    m = re.search(r"audit\s*score\s*(?:<=|<|under|below)\s*(\d+)", ql)
    if m: f["max_audit_score"] = int(m.group(1))
    # ESG keywords
    esg=[]
    if "low emission" in ql or "low-emission" in ql: esg.append("low_emissions")
    if "conflict-free" in ql or "conflict free" in ql: esg.append("conflict_free")
    if "recycled" in ql: esg.append("recycled_materials")
    if esg: f["esg"]=esg
    # Risk
    m = re.search(r"risk\s*score\s*(?:<=|<)\s*(\d+)", ql)
    if m: f["risk_lte"]=int(m.group(1))
    f["need_liability"]=("liability cap" in ql)
    f["need_ddp"]=("incoterms" in ql or "ddp" in ql)
    return f

def passes(md: dict, f: dict) -> bool:
    ok=True
    if "region" in f:
        ok &= (md.get("region")==f["region"] or f["region"] in str(md.get("region","")))
    if "certs" in f and md.get("doctype")=="audit":
        if "ISO13485" in f["certs"]: ok&=bool(md.get("iso13485",0))
        if "ROHS" in f["certs"]    : ok&=bool(md.get("rohs",0))
    if "max_lt" in f:
        lt=md.get("lead_time_weeks"); ok &= (lt is None) or (isinstance(lt,(int,float)) and lt<=f["max_lt"])
    # Price filters (for pricing docs)
    if md.get("doctype") == "pricing":
        if "max_price" in f:
            price = md.get("price_usd")
            if price is not None:
                ok &= float(price) <= f["max_price"]
        if "min_price" in f:
            price = md.get("price_usd")
            if price is not None:
                ok &= float(price) >= f["min_price"]
    # Audit score filters (for audit docs)
    if md.get("doctype") == "audit":
        if "min_audit_score" in f:
            score = md.get("score")
            if score is not None:
                ok &= float(score) >= f["min_audit_score"]
        if "max_audit_score" in f:
            score = md.get("score")
            if score is not None:
                ok &= float(score) <= f["max_audit_score"]
    if "esg" in f:
        flags = str(md.get("esg_flags","")).lower()
        ok &= all(tag.replace("_"," ") in flags for tag in f["esg"])
    if "risk_lte" in f and md.get("doctype") in ["profile"]:
        rs = md.get("risk_score"); ok &= (isinstance(rs,(int,float)) and rs<=f["risk_lte"])
    if f.get("need_liability"): ok &= (md.get("doctype")=="contract")
    if f.get("need_ddp"):       ok &= (md.get("doctype") in ["contract","capability","pricing","profile"])
    return ok

## Hybrid retrieval (BM25 + FAISS) + LLM re-ranking + evidence fusion

In [12]:
# --- Hybrid retrieval: FAISS + BM25, simple score fusion ---
def faiss_search_with_scores(query: str, k: int = 40):
    # langchain FAISS returns (Document, score) where score is L2 distance by default
    res = vs.similarity_search_with_score(query, k=k)
    # Convert distance to relevance in [0,1] (smaller distance → higher relevance)
    # We normalize across this batch.
    dists = [s for _,s in res]
    if not dists: return []
    dmin, dmax = min(dists), max(dists)
    if dmax == dmin: dmax += 1e-6
    pairs = []
    for doc, dist in res:
        rel = 1.0 - (dist - dmin)/(dmax - dmin)  # 1 = best
        pairs.append((doc, rel))
    return pairs

def bm25_search_with_scores(query: str, k: int = 40):
    scores = bm25.get_scores(_tokenize(query))
    # normalize to [0,1]
    smin, smax = float(min(scores)), float(max(scores))
    if smax == smin: smax += 1e-6
    idx = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:k]
    return [(docs[i], (scores[i]-smin)/(smax-smin)) for i in idx]

# Keep a reference list 'docs' indexed same as bm25_texts
docs = docs  # already created earlier

def hybrid_retrieve(query: str, k_faiss=40, k_bm25=40, alpha: float = 0.55, top_k: int = 40):
    """
    alpha = weight for FAISS; (1-alpha) for BM25. Returns a list of Documents (no duplicates)
    sorted by fused relevance.
    """
    f = faiss_search_with_scores(query, k=k_faiss)
    b = bm25_search_with_scores(query, k=k_bm25)

    # Accumulate scores by doc id (use content+supplier_id as signature)
    def sig(d: Document): return (d.page_content, tuple(sorted(d.metadata.items())))
    scores = {}
    for d, s in f:
        scores.setdefault(sig(d), {"doc": d, "faiss": 0.0, "bm25": 0.0})
        scores[sig(d)]["faiss"] = max(scores[sig(d)]["faiss"], s)
    for d, s in b:
        scores.setdefault(sig(d), {"doc": d, "faiss": 0.0, "bm25": 0.0})
        scores[sig(d)]["bm25"] = max(scores[sig(d)]["bm25"], s)

    fused = []
    for key, val in scores.items():
        fused_score = alpha*val["faiss"] + (1-alpha)*val["bm25"]
        fused.append((val["doc"], fused_score))
    fused.sort(key=lambda x: -x[1])
    return [d for d,_ in fused[:top_k]]

# --- LLM re-ranker (compact scoring, keeps top_k) ---
from textwrap import dedent
_rank_llm = ChatOpenAI(model=SET.chat_model, temperature=0)

def _truncate(t: str, n: int = 600) -> str:
    t = " ".join(t.split())
    return t if len(t) <= n else t[:n-3]+"..."

def llm_rerank(query: str, docs_list: list[Document], top_k: int = 6) -> list[Document]:
    if not docs_list: return []
    items = []
    for i,d in enumerate(docs_list,1):
        hint=f" [type={d.metadata.get('doctype','')}, supplier={d.metadata.get('supplier_id','')}]"
        items.append(f"{i}. {_truncate(d.page_content)}{hint}")
    prompt = dedent(f"""
    Score each item from 0 to 100 for relevance to the query. Return JSON {{"scores":[...]}} only.

    Query:
    {query}

    Items:
    {chr(10).join(items)}
    """).strip()
    resp = _rank_llm.invoke([{"role":"user","content":prompt}]).content.strip()
    try:
        j = json.loads(resp[resp.find("{"):resp.rfind("}")+1])
        scores = j.get("scores", [])
        ranked = sorted(zip(scores, docs_list), key=lambda x: -x[0])
        return [d for _,d in ranked[:top_k]]
    except Exception:
        return docs_list[:top_k]

# --- Evidence fusion: ensure explicit lead-time facts in context ---
_PRICING_PATH = RAW/"pricing"/"pricing.csv"
_PRICING_DF = pd.read_csv(_PRICING_PATH)

def pricing_rows_for_supplier(supplier_id: str, max_lt: int|None=None, limit=2) -> list[Document]:
    df = _PRICING_DF[_PRICING_DF["supplier_id"]==supplier_id].copy()
    if max_lt is not None: df = df[df["lead_time_weeks"]<=max_lt]
    if df.empty: return []
    df = df.sort_values(["lead_time_weeks","price_usd"]).head(limit)
    out=[]
    for _,r in df.iterrows():
        text=(f"Pricing: supplier {r['supplier_id']} sku {r['sku']} "
              f"lead_time_weeks {int(r['lead_time_weeks'])} price_usd {r['price_usd']} "
              f"region {r['region']} category {r['category']}")
        out.append(Document(page_content=text, metadata={
            "supplier_id": r["supplier_id"], "doctype":"pricing", "sku": r["sku"],
            "lead_time_weeks": int(r["lead_time_weeks"]), "region": r["region"], "category": r["category"]
        }))
    return out

## Compact answering chain and console-style printer

In [13]:
# Generator prompt (bullets, grounded)
SIMPLE_SYSTEM = """You are Sourcing and procurement assistant.
Use ONLY the provided context. Answer in short bullet points.
Include when available: Supplier ID/Name, Region, Lead time (weeks),
Certifications (ISO13485/ROHS), Contract clauses (Liability Cap, Incoterms DDP),
Audit score, and OTD. If something is missing, write what is missing."""
_prompt = ChatPromptTemplate.from_messages([
    ("system", SIMPLE_SYSTEM),
    ("human", "Question: {q}\n\nContext:\n{ctx}")
])
_gen_llm = ChatOpenAI(model=SET.chat_model, temperature=0.1)
qa_chain = _prompt | _gen_llm | StrOutputParser()

# Pretty formatting helpers
from textwrap import fill

def wrap(text: str, width: int = 86) -> str:
    return "\n".join(fill(l, width=width) if l.strip() else "" for l in (text or "").splitlines()).rstrip()

def sep(width: int = 86, ch: str = "=") -> str:
    return ch * width

def make_sources_block(sources: list[dict], width: int = 86, max_chars: int = 220) -> str:
    out=[]
    for i,s in enumerate(sources,1):
        md=s.get("metadata",{})
        sid=md.get("supplier_id","—"); dtype=md.get("doctype","doc")
        snippet=(s.get("content") or "").replace("\n"," ")
        if len(snippet)>max_chars: snippet=snippet[:max_chars-3]+"..."
        out.append(fill(f"[{i}] ({dtype}) {sid} — {snippet}", width=width))
    return "\n".join(out).rstrip()

## End-to-end pipeline: Hybrid retrieval → filters → rerank → fusion → answer

In [14]:
# Load FAISS from disk (handy if re-running later)
vectorstore = FAISS.load_local(SET.index_dir, embeddings=emb, allow_dangerous_deserialization=True)

def ask_pretty(
    question: str,
    width: int = 86,
    k_hybrid: int = 40,   # total candidates after hybrid fusion
    k_post: int = 16,     # after client filters
    k_final: int = 8,     # after LLM rerank (context size)
    show_sources: bool = True,
    print_output: bool = True,   # <--- NEW: control printing
    export_excel_path: str = None,  # <--- NEW: path to export LLM answer to Excel
):
    # 1) Hybrid retrieve (BM25 + FAISS → fused list)
    prelim = hybrid_retrieve(question, top_k=k_hybrid)

    # 2) Client-side filters
    f = extract_filters(question)
    filtered = [d for d in prelim if passes(d.metadata, f)]
    cand = (filtered[:k_post] if filtered else prelim[:k_post])

    # 3) Pricing boost if user constrains lead time
    if "max_lt" in f:
        extra = vectorstore.similarity_search("pricing lead time weeks", k=12)
        extra = [d for d in extra if d.metadata.get("doctype") == "pricing"]
        seen = {d.page_content for d in cand}
        for d in extra:
            if d.page_content not in seen:
                cand.append(d); seen.add(d.page_content)

    # 4) LLM rerank
    topk = llm_rerank(question, cand, top_k=k_final)

    # 5) Evidence fusion: guarantee LT presence
    sids = sorted({d.metadata.get("supplier_id") for d in topk if d.metadata.get("supplier_id")})
    fused = list(topk)
    seen_txt = {d.page_content for d in fused}
    for sid in sids:
        for doc in pricing_rows_for_supplier(sid, max_lt=f.get("max_lt"), limit=2):
            if doc.page_content not in seen_txt:
                fused.append(doc); seen_txt.add(doc.page_content)

    # 6) Build context & answer (bullets)
    ctx = "\n".join(f"- {d.page_content}" for d in fused)
    answer_text = qa_chain.invoke({"q": question, "ctx": ctx})

    # 7) Compose a single pretty string (so export can save it)
    header = f"{sep(width)}\n**User Query:**\n{wrap(question, width)}\n\n"
    body   = f"{sep(width)}\n**Generated Response:**\n\n{wrap(answer_text, width)}\n\n{sep(width)}"
    srcs = []
    pretty = header + body
    if show_sources:
        srcs = [{"content": d.page_content, "metadata": d.metadata} for d in fused[:10]]
        sources_block = make_sources_block(srcs, width=width)
        pretty += f"\n\n**Sources:**\n{sources_block}"

    # 8) Print only if requested
    if print_output:
        print(pretty)

    # 9) Export LLM answer to Excel if requested
    if export_excel_path is not None:
        import pandas as pd
        # Each bullet point as a row, remove markdown formatting
        answer_lines = [line.strip('- ').strip() for line in answer_text.split('\n') if line.strip()]
        df = pd.DataFrame({'LLM_Answer': answer_lines})
        df.to_excel(export_excel_path, index=False)
        print(f"LLM answer exported to Excel: {export_excel_path}")

    # Return both raw and pretty strings for downstream use
    return {
        "question": question,
        "answer_text": answer_text,   # raw LLM bullets
        "pretty": pretty,             # console-style block
        "sources": srcs,
    }

## Test Demo (prints final outputs)

In [15]:
tests = [
    "Alternative suppliers in ASEAN with lead time <= 4 weeks and ISO 13485",
    "Which contracts mention liability cap and Incoterms DDP?",
    "Show EU suppliers for bearings with ROHS compliance and low risk (risk score < 40)",
    "Suppliers with best audit scores, short lead time, and OTD > 90%",
]

print("\n=== TEST START ===")
for i, q in enumerate(tests, 1):
    print(f"\n# Test {i}/{len(tests)}")
    ask_pretty(q, width=86)

print("\n Test finished.")


=== TEST START ===

# Test 1/4
**User Query:**
Alternative suppliers in ASEAN with lead time <= 4 weeks and ISO 13485

**Generated Response:**

- Supplier ID/Name: S1006
  - Region: ASEAN
  - Lead time: 3 weeks
  - Certifications: ISO 13485 (missing confirmation)
  - Contract clauses: (missing)
  - Audit score: (missing)
  - OTD: (missing)

- Supplier ID/Name: S1009
  - Region: ASEAN
  - Lead time: 4 weeks
  - Certifications: ISO 13485 (missing confirmation)
  - Contract clauses: (missing)
  - Audit score: (missing)
  - OTD: (missing)


**Sources:**
[1] (pricing) S1006 — Pricing: supplier S1006 sku BRG-6205 lead_time_weeks 3 price_usd
11.75 region ASEAN
[2] (pricing) S1009 — Pricing: supplier S1009 sku BRG-6205 lead_time_weeks 4 price_usd
50.63 region ASEAN
[3] (pricing) S1009 — Pricing: supplier S1009 sku IMP-75 lead_time_weeks 5 price_usd
74.17 region ASEAN
[4] (pricing) S1006 — Pricing: supplier S1006 sku IMP-75 lead_time_weeks 8 price_usd
23.76 region ASEAN
[5] (pricing) S1006 — P

## Export LLM Answer to Excel

In [16]:
tests = [
    "Alternative suppliers in ASEAN with lead time <= 4 weeks and ISO 13485",
    "Which contracts mention liability cap and Incoterms DDP?",
    "Show EU suppliers for bearings with ROHS compliance and low risk (risk score < 40)",
    "Suppliers with best audit scores, short lead time, and OTD > 90%",
]

for i, q in enumerate(tests, 1):
    excel_path = f"llm_answer_{i}.xlsx"
    print(f"\nRunning test {i}: {q}")
    ask_pretty(q, export_excel_path=excel_path, print_output=False)
    print(f"Saved LLM answer to: {excel_path}")

print("\nAll test answers exported to Excel. You can open the .xlsx files in Excel or Google Sheets.")


Running test 1: Alternative suppliers in ASEAN with lead time <= 4 weeks and ISO 13485
LLM answer exported to Excel: llm_answer_1.xlsx
Saved LLM answer to: llm_answer_1.xlsx

Running test 2: Which contracts mention liability cap and Incoterms DDP?
LLM answer exported to Excel: llm_answer_2.xlsx
Saved LLM answer to: llm_answer_2.xlsx

Running test 3: Show EU suppliers for bearings with ROHS compliance and low risk (risk score < 40)
LLM answer exported to Excel: llm_answer_3.xlsx
Saved LLM answer to: llm_answer_3.xlsx

Running test 4: Suppliers with best audit scores, short lead time, and OTD > 90%
LLM answer exported to Excel: llm_answer_4.xlsx
Saved LLM answer to: llm_answer_4.xlsx

All test answers exported to Excel. You can open the .xlsx files in Excel or Google Sheets.
